Import Classes and Define Variables

In [1]:
import numpy as np
import scipy as sp
import pymongo
import pandas as pd
import nsepy
from datetime import datetime
from datetime import timedelta
from pymongo import MongoClient
from nsepy import get_history
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]

8425

Data Collector Function

In [2]:
def dataCollector():
    equityListPd = pd.DataFrame(equityList.find({'State':"A"}))
    collectionTime=2000
    basicSample=30
    cnt=0
    my_time = datetime.now()

    for rows in equityListPd['Issuer Name'].iteritems():
        print(cnt,rows[1],end="-")
        cnt= cnt + 1
        if stockWarehouse.find({'Symbol':rows[1]}).count() > 1 :
            for j in stockWarehouse.find({'Symbol':rows[1]},{'_id':0,'Date':1}).sort('Date',-1).limit(1) :
                collectionTime=((my_time - j['Date']).days - 1)
                fromDate=j['Date']
        if fromDate.date() < datetime.now().date():
            
            iters=int(collectionTime/basicSample)
            if (collectionTime != 0):
                if collectionTime < 2000:
                    basicSample=collectionTime
                    iters=1
                dt=datetime.now()
                print("Iterations:",iters,end="|")
                for i in range(0,iters):

                    startDate=dt - timedelta(days=basicSample)
                    if( startDate <= fromDate):
                        startDate = (fromDate + timedelta(days=1))
                    endDate=dt 
                    print(i,"Collecting into between",startDate.date(),"-", endDate.date(),"@",datetime.now(),end="|")
                    df=pd.DataFrame(get_history(symbol=rows[1] , start= startDate,end=endDate))
                    df.reset_index(inplace=True)
                    df['Date']=pd.to_datetime(df['Date'])
                    dfLength=(len(df.index))
                    if  dfLength > 0 :
                        data_dict=df.to_dict("records")
                        #print(data_dict)
                        stockWarehouse.insert_many(data_dict)
                        tmpdf=pd.DataFrame(stockWarehouse.find({"Symbol":rows[i],"Date":{"$lte":endDate,"$gte":startDate}}))
                        print(df.index.size,tmpdf.index.size,end="|")
                        equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Active'}})    
                    else :
                        print("Marking as Inactive")
                        equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Inactive'}})
                        break;
                    dt=(startDate - timedelta(days=1))


In [3]:
def markInactiveStocks():
    reference=int(stockWarehouse.find({'Symbol':'SBIN'}).count()) - 100
    print(reference)
    tmp_count=0
    for i in equityList.find({},{'Issuer Name':1}):
        #print(i['Issuer Name'])
        tmp_count=(int(stockWarehouse.find({'Symbol':i['Issuer Name']}).count()))
        if tmp_count >= reference:
            print(i['Issuer Name'],end='|')
            equityList.update_many({'Issuer Name':i['Issuer Name']},{'$set': {'State':"A"}})

In [4]:
def markTrend_Crossover(stockdf,datadf,symbol):
    
    for i in range(0,stockdf.shape[0],1):
            if stockdf['Issuer Name'].iloc[i]==symbol:
                break;
    datadf['ewmdiff']=datadf['Close'].ewm(span=9,adjust=False).mean() - datadf['Close'].ewm(span=21,adjust=False).mean()
    nrows=datadf.shape[0]-1
    if datadf.iloc[nrows,15] > 0:
        stockdf.iloc[i,1]=1
    else:
        stockdf.iloc[i,1]=-1
    for j in range(datadf.shape[0]-1,2,-1):
        if datadf.iloc[j,15]*datadf.iloc[j-1,15] <= 0:
            stockdf.iloc[i,2]=datadf.shape[0]-j
            break;
    datadf['ewmdiffewm14']=datadf['ewmdiff'].ewm(span=9,adjust=False).mean()
    if datadf.iloc[nrows,15] > datadf.iloc[nrows,16] :
        stockdf.iloc[i,3]='Expanding'
    else:
        stockdf.iloc[i,3]='Contracting'
    return(stockdf)

In [5]:
def singlePatternCandle(stockdf,datadf,symbol):
    for i in range(0,stockdf.shape[0],1):
        if stockdf['Issuer Name'].iloc[i]==symbol:
            break;
                
    pattern=''
    umbrellaFound=False
    if 'singleCandlePattern' in stockdf.columns:
        if pd.isnull(stockdf['singleCandlePattern'].iloc[i]):
            pattern=''
        else:
            pattern=stockdf['singleCandlePattern'].iloc[i]
    nrows=datadf.shape[0]-1
    Close=datadf['Close'].iloc[nrows]
    Open=datadf['Open'].iloc[nrows]
    High=datadf['High'].iloc[nrows]
    Low=datadf['Low'].iloc[nrows]
    if  Open > Close :
        candle=-1
    if  Open < Close :
        candle=1
    if  Open == Close :
        candle=0
####### Check MaruBozo
    
    if candle > 0:
        if ((High-Close)/(Close-Open)) < .05 and ((Open-Low)/(Close-Open))< .05:
            pattern=pattern + 'Bull Marubozo|' +'SL:' + str(Low)
    if candle < 0:
        if ((High-Open)/(Open-Close)) < .05 and ((Close-Low)/(Open-Close)) < .05:
            pattern=pattern + 'Bear Marubozo|' +'SL:' + str(High)
        
####### Check Paper Umbrella
    if candle > 0:
        if ((Open-Low)/(Close-Open)) > 2 and (High-Close)/(Close-Open) < .05:
            umbrellaFound=True
    if candle < 0:
        if ((Close-Low)/(Open-Close)) > 2 and (High-Open)/(Open-Close) < .05:
            umbrellaFound=True
    if umbrellaFound:
        if stockdf['trend'].iloc[i] == 1:
            pattern=pattern + 'Hanging Man ' +'SL:' + str(High)
        else:
            pattern=pattern + 'Hammer ' + 'SL:' + str(Low)
        
    stockdf['singleCandlePattern'].iloc[i]=pattern
    return(stockdf)
                
        
        

In [6]:
def rsiindicator(stockdf,datadf,symbol,term):
    for i in range(0,stockdf.shape[0],1):
        if stockdf['Issuer Name'].iloc[i]==symbol:
            break;
    nrows=datadf.shape[0]-1
    datadf['Diff']=datadf['Close'].diff()
    datadf['AVGU']=0
    datadf['AVGD']=0
    datadf.loc[datadf['Diff'] > 0,'AVGU' ] = datadf['Diff']
    datadf.loc[datadf['Diff'] < 0,'AVGD' ] = -datadf['Diff']
    AVGUT=datadf.iloc[nrows-1,18]/term + (term - 1)*datadf.iloc[(nrows-(term+1)):(nrows-2),18].mean()/term    
    AVGDT=datadf.iloc[nrows-1,19]/term + (term - 1)*datadf.iloc[(nrows-(term+1)):(nrows-2),19].mean()/term
    RS=AVGUT/AVGDT
    RSI=100-(100/(1+RS))
    stockdf.iloc[i,5]=RSI
    return(stockdf)

In [7]:
dataCollector()

0 ABB-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:40.000841|

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()


Marking as Inactive
1 AMARAJABAT-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:41.394780|Marking as Inactive
2 HDFC-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:41.621558|Marking as Inactive
3 ANSALAPI-Iterations: 1|0 Collecting into between 2020-05-07 - 2020-08-03 @ 2020-08-03 11:36:41.936004|Marking as Inactive
4 ASIANHOTNR-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:42.355062|Marking as Inactive
5 ATUL-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:42.749920|Marking as Inactive
6 AUTOLITIND-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:42.985391|Marking as Inactive
7 BAJAJHIND-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:43.232934|Marking as Inactive
8 BAJFINANCE-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:36:43.485840|Marking

67 INGERRAND-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:37:01.778188|Marking as Inactive
68 ATFL-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:37:02.024557|Marking as Inactive
69 JISLJALEQS-Iterations: 1|0 Collecting into between 2020-02-29 - 2020-08-03 @ 2020-08-03 11:37:02.256546|33 0|70 JINDALPOLY-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:37:02.691700|Marking as Inactive
71 JSWSTEEL-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:37:02.933899|Marking as Inactive
72 KAJARIACER-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:37:03.171440|Marking as Inactive
73 KAKATCEM-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:37:03.405539|Marking as Inactive
74 KSL-Iterations: 1|0 Collecting into between 2020-08-01 - 2020-08-03 @ 2020-08-03 11:37:03.668250|Marking as Inactive
75 WHIRLPOOL-Iter

134 UCALFUEL-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:28.311197|1 0|135 HARRMALAYA-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:28.564204|1 0|136 FEDERALBNK-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:28.807236|1 0|137 TATASTEEL-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:29.047119|1 0|138 SKFINDIA-Iterations: 1|0 Collecting into between 2020-07-04 - 2020-08-03 @ 2020-08-03 11:37:29.278892|20 0|139 ASHOKLEY-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:29.557121|1 0|140 CUMMINSIND-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:29.807157|1 0|141 TATACOMM-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:30.040522|1 0|142 ABBOTINDIA-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:37:30.297061|1 0|

...

556 GRANULES-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:51.242789|1 0|557 CANBK-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:51.481515|1 0|558 DIVISLAB-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:51.712529|1 0|559 ALBK-Iterations: 1|0 Collecting into between 2020-02-29 - 2020-08-03 @ 2020-08-03 11:39:51.955707|13 0|560 UBL-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:52.803045|1 0|561 ASTRAMICRO-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:53.030959|1 0|562 RADICO-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:53.266989|1 0|563 SHRIRAMCIT-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:53.508009|1 0|564 MARUTI-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:39:53.734947|1 0|565 UCOBANK-Iteration

630 NITCO-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:15.210441|1 0|631 SOLARINDS-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:15.454201|1 0|632 UTTAMSUGAR-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:16.016388|1 0|633 SUNTV-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:16.264394|1 0|634 GPIL-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:16.611978|1 0|635 EMKAY-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:16.923140|1 0|636 LOKESHMACH-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:17.196932|1 0|637 PFOCUS-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:17.558527|1 0|638 ALLCARGO-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:17.827249|1 0|639 VOLTAMP-Iteratio

679 DELTACORP-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:37.123330|1 0|680 INSECTICID-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:37.340464|1 0|681 MCDHOLDING-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:37.819134|1 0|682 ASAHISONG-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:38.056960|1 0|683 TIMETECHNO-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:38.450406|1 0|684 HGS-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:38.689258|1 0|685 NELCAST-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:39.071021|1 0|686 MEGH-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:39.299880|1 0|687 DLF-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:39.547176|1 0|688 TARMAT-Iterat

747 DLINKINDIA-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:58.752046|1 0|748 JSWENERGY-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:59.132903|1 0|749 GODREJPROP-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:59.511855|1 0|750 DBCORP-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:40:59.878342|1 0|751 JUBLFOOD-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:00.666699|1 0|752 VASCONEQ-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:01.065922|1 0|753 THANGAMAYL-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:01.874846|1 0|754 EMMBI-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:02.272179|1 0|755 HATHWAY-Iterations: 1|0 Collecting into between 2020-07-23 - 2020-08-03 @ 2020-08-03 11:41:02.743195|Marking as In

819 SPECIALITY-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:30.790249|1 0|820 LYPSAGEMS-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:31.167718|1 0|821 LAKSHVILAS-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:31.553815|1 0|822 ZUARI-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:31.773772|1 0|823 CARERATING-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:32.111747|1 0|824 PCJEWELLER-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:32.678532|1 0|825 INFRATEL-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:33.050511|1 0|826 VMART-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:33.286521|1 0|827 REPCOHOME-Iterations: 1|0 Collecting into between 2020-07-31 - 2020-08-03 @ 2020-08-03 11:41:33.669498|1 0|828 JU

In [8]:
markInactiveStocks()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  import sys


1357
ABB|AMARAJABAT|HDFC|ANSALAPI|ASIANHOTNR|ATUL|AUTOLITIND|BAJAJHIND|BAJFINANCE|BALRAMCHIN|BANCOINDIA|CENTURYTEX|BASF|BATAINDIA|BANARISUG|BEL|BEML|BEPL|BLUESTARCO|BPL|CENTEXT|CESC|CHAMBLFERT|EXIDEIND|CIPLA|DICIND|CRISIL|DABUR|DALMIASUG|ARVIND|BALLARPUR|BHEL|HINDPETRO|IFCI|MTNL|MRPL|CHENNPETRO|RELCAPITAL|SAIL|TITAN|IDBI|DCW|DHAMPURSUG|DRREDDY|EIDPARRY|ELECTCAST|SBIN|ESABINDIA|UFLEX|FOSECOIND|GODREJIND|KANSAINER|GHCL|HCL-INSYS|HDFCBANK|HEROMOTOCO|HFCL|HCC|HINDOILEXP|HSIL|HINDZINC|INDIAGLYCO|INDORAMA|INFY|INGERRAND|ATFL|JISLJALEQS|JINDALPOLY|JSWSTEEL|KAJARIACER|KAKATCEM|KSL|WHIRLPOOL|KIRLOSBROS|KIRLOSIND|KOTAKBANK|LGBBROSLTD|TRENT|LAXMIMACH|LICHSGFIN|LUPIN|LYKALABS|RAMCOCEM|MAHSEAMLES|MAHSCOOTER|MANALIPETC|MIRCELECTR|CENTENKA|MOREPENLAB|MRF|HEIDELBERG|NCC|NAHARSPING|GRASIM|PEL|NIITLTD|ONGC|ORIENTHOT|ORIENTBANK|RELIANCE|BIRLACORPN|PIDILITIND|PRIMESECU|RAYMOND|SURYAROSNI|RSWM|RAMANEWS|RALLIS|JINDALSAW|JKLAKSHMI|SHREECEM|RELINFRA|TATAPOWER|SPMLINFRA|SUNDRMFAST|SUNFLAG|SUPPETRO|SWARAJENG|TA

In [10]:
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListdf=pd.DataFrame(equityList.find({'State':"A"},{'Issuer Name':1,'_id':0}))
equityListdf['trend']=''
equityListdf['daysSinceCutOver']=''
equityListdf['trendnature']=''
equityListdf['singleCandlePattern']=''
equityListdf['RSI']=''
equityListdf['Close']=0
equityListdf['Signal']=0
           
for dt in stockWarehouse.find({'Symbol':'SBIN'},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
    refdate=dt # Set Reference Data as last date when SBI Stock Data was update
print(refdate)
i=0
for symbol in equityListdf['Issuer Name']    :
    for maxdate in stockWarehouse.find({'Symbol':symbol},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
        print(symbol,maxdate)
        if maxdate == refdate:  # Check if database has latest sttock info by comparing aginst SBI
            datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol}).sort('Date',-1).limit(300))# Get top 300 row of stock in iteration
            datadf.set_index('Date',inplace=True)
            datadf.sort_index(inplace=True)
            equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Close']=datadf.iloc[-1,1]
            equityListdf=markTrend_Crossover(equityListdf,datadf,symbol)
            equityListdf=singlePatternCandle(equityListdf,datadf,symbol)
            equityListdf=rsiindicator(equityListdf,datadf,symbol,15)
            tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()    
#            print(equityListdf.columns)
            datadf['avgvolume30']=datadf['Volume'].rolling(30).mean()
            datadf['avgvolume10']=datadf['Volume'].rolling(10).mean()
                 
            if tempnp[0,5] < 45 and tempnp[0,2] < 20 and tempnp[0,5] > 20  and tempnp[0,1] == 1 and tempnp[0,3] == 'Expanding'  and datadf['avgvolume30'].iloc[-1]<datadf['avgvolume10'].iloc[-1] :
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Signal']=1
            else :
                if (tempnp[0,5] > 80 or tempnp[0,1] == -1 or tempnp[0,3] == 'Contracting'):
                    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Signal']=-1

            
            
equityListdf.loc[equityListdf['Signal']>0].to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\Buy.csv")
equityListdf.loc[equityListdf['Signal']<0].to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\sell.csv")
equityListdf.to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\result.csv")
print("Buy")



{'Date': datetime.datetime(2020, 7, 31, 0, 0)}
ABB {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
AMARAJABAT {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
HDFC

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


 {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
ANSALAPI {'Date': datetime.datetime(2020, 5, 6, 0, 0)}
ASIANHOTNR {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
ATUL

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


 {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
AUTOLITIND {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
BAJAJHIND {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
BAJFINANCE {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
BALRAMCHIN

...

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


KOTAKPSUBK {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
PSUBNKBEES {'Date': datetime.datetime(2020, 7, 31, 0, 0)}


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


M50 {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
MADRASFERT {'Date': datetime.datetime(2020, 7, 31, 0, 0)}
Buy


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
equityListdf.loc[equityListdf['Signal']<0]

,Issuer Name,trend,daysSinceCutOver,trendnature,singleCandlePattern,RSI,Close,Signal
0,ABB,-1,3,Contracting,,28.3415,894.95,-1
1,AMARAJABAT,1,71,Contracting,,56.0644,709.20,-1
2,HDFC,1,44,Contracting,,44.1211,1781.95,-1
4,ASIANHOTNR,-1,5,Contracting,,35.8663,54.15,-1
5,ATUL,1,20,Expanding,,82.9766,5053.95,-1
7,BAJAJHIND,-1,15,Contracting,,24.2424,5.60,-1
8,BAJFINANCE,1,43,Contracting,,48.421,3251.30,-1
9,BALRAMCHIN,-1,1,Contracting,,49.4792,125.50,-1
10,BANCOINDIA,-1,1,Contracting,,41.7582,79.00,-1
11,CENTURYTEX,-1,6,Contracting,,42.0015,307.95,-1
